In [536]:
import numpy as np

In [537]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   웨이브 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 웨이브 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

batch_size 개수만큼의 시계열 데이터가 생성

각 시계열 데이터는 n_steps 만큼의 데이터가 있고,

앞선 n_steps - 1 개의 데이터를 통해 n_steps 번째의 데이터를 예측한다.

In [538]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [539]:
X_train.shape

(7000, 50, 1)

In [540]:
X_train[0].shape

(50, 1)

In [541]:
y_train.shape

(7000, 1)

In [542]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):
    return x * (1 - x)

In [543]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self):
    return (self.predict[0][0] - self.target[0][0])

In [544]:
class RNN_vector_to_sequence:
    #입력값
    input_data = []
    
    #가중치
    weight = []
    
    #편향값
    bias = []

    # 입력 가중치
    input_w = []

    # 이전 출력에 대한 가중치
    before_w = []

    # 편향 값 bais
    bias = []

    # 노드 입력
    node_input = []

    # 노드 출력
    node_output = []

    # 타겟, 목푯값
    target = []

    # 활성화 함수
    activation = activation_function()

    # 비용 함수
    cost = cost_function()

    # 예측 결과
    predict = []

    # 각 층의 델타 값의 저장
    delta = []

     # 각 가중치 업데이트 크기
    input_weight_update = []

    before_weight_update = []

    def delta_to_sequence_cal_rnn_test(self, input, target):

      # 타겟값의 저장
      self.target = target

      # 입력 데이터의 저장
      self.input = input

      # 입력 데이터 크기에 맞는 가중치 값이 생성되어야 한다.
      # 입력 (3,1) 데이터에 대한 연산 결과 (1,1)를 예측한다고 가정,

      # 입력값에 사용되는 가중치
      # (3,1) 크기의 입력 데이터에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다
      self.input_w = np.random.rand(target.shape[0], input.shape[1])
      
      # 이전 출력에 대한 가중치
      # (1,1) 노드 출력에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위한, (1,1) 크기의 가중치 행렬
      self.before_w = np.random.rand(target.shape[0], target.shape[0])

      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((target.shape[0], target.shape[0]))

      self.node_output.append(before)
      
      # (3, n) 크기의 입력 데이터에서 n 번의 반복을 수행하게 됨
      for i in range(input.shape[0]):
        # 노드 입력 데이터의 numpy 행렬 생성
        input_data = input[i].reshape(input.shape[1],-1)

        # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
        node_input = (self.before_w @ before) + (self.input_w @ input_data)
        self.node_input.append(node_input)

        # 노드 출력 계산, 활성화 함수 연산을 수행한다.
        node_output = self.activation.sigmoid(node_input)
        self.node_output.append(node_output)

        # 타입 스텝의 출력이 다음 노드의 입력이 된다.
        before = node_output

      self.predict = node_output

      self.cost_cal()

      return self.predict

    def cost_cal(self):
      return self.cost.errer_squared_sum(self.predict, self.target)

    
    def cal_delta(self):
      #마지막 노드의 변화량에 대한 오차 함수의 변화량 계산
      delta = (self.cost.diff_error_squared_sum() * self.activation.sigmoid_diff(self.predict))
      self.delta.append(delta)

      # n개의 delta 값을 계산하기 위한 반복
      for i in range(input.shape[0] - 1, -1, -1):
        # 노드 출력의 변화에 대한 비용 함수의 변화량 계산
        delta = self.before_w @ delta
        # 활성화 함수에 따른 변화량 계산, 해당 값이 노드 변화에 대한 비용 함수의 변화량이 된다. 
        delta = delta * self.activation.sigmoid_diff(self.node_output[self.input.shape[0] - 1])
        # 노드별 delta 값의 저장
        self.delta.append(delta)
    

    def update_weight(self, learning_rate):
      # 원활한 연산을 위해 마지막 노드부터 거꾸로 계산된 delta 값을 뒤집어준다.
      self.delta = self.delta[::-1]

      result = 0

      # 해당 delta 값과의 데이터 입력값을 통해 input_w 의 가중치 변화량을 계산할 수 있다
      for i in range(self.input.shape[0]):
        result = result + self.delta[i] * self.node_input[i]

      self.before_weight_update = result

      # 이전 노드 출력값을 통해 before_w 의 가중치 변화량 계산
      result = 0

      for i in range(self.input.shape[0]):
        result = result + self.delta[i] * self.input[i]

      self.input_weight_update = result

      # 가중치 업데이트
      self.input_w = self.input_w - (self.input_weight_update * learning_rate)

      self.before_w = self.before_w - (self.before_weight_update * learning_rate)

    def iterations(self, iterations, learning_rate):
      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((self.target.shape[0], self.target.shape[0]))
      for j in range(iterations):
        # 초기화
        self.node_input = []
        self.node_output = []
        for i in range(input.shape[0]):
          # 노드 입력 데이터의 numpy 행렬 생성
          input_data = input[i].reshape(input.shape[1],-1)

          # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
          node_input = (self.before_w @ before) + (self.input_w @ input_data)
          self.node_input.append(node_input)

          # 노드 출력 계산, 활성화 함수 연산을 수행한다.
          node_output = self.activation.sigmoid(node_input)
          self.node_output.append(node_output)

          # 순환 노드의 다음 노드 입력값의 계산
          before = node_output

        self.predict = before
        self.cal_delta()
        self.update_weight(learning_rate)
        print(self.cost_cal(), self.predict, self.target)



In [545]:
rnn = RNN_vector_to_sequence()

In [546]:
input = X_train[0]

In [547]:
target = y_train[0].reshape(1,-1)

In [548]:
input.shape

(50, 1)

In [549]:
target.shape

(1, 1)

In [550]:
rnn.delta_to_sequence_cal_rnn_test(input, target)

array([[0.54564968]])

In [551]:
rnn.cal_delta()

In [552]:
rnn.iterations(1000, 10)

0.4054894435206465 [[0.54564968]] [[-0.35489398]]
0.3959451604343717 [[0.53498821]] [[-0.35489398]]
0.38942254407455557 [[0.52762802]] [[-0.35489398]]
0.3846538080898872 [[0.52220785]] [[-0.35489398]]
0.380960783523163 [[0.5179872]] [[-0.35489398]]
0.37798792680455673 [[0.51457474]] [[-0.35489398]]
0.37552706738648656 [[0.51173981]] [[-0.35489398]]
0.37344648449372475 [[0.50933571]] [[-0.35489398]]
0.3716579668784316 [[0.50726374]] [[-0.35489398]]
0.37009980898983896 [[0.50545456]] [[-0.35489398]]
0.36872732160902205 [[0.50385781]] [[-0.35489398]]
0.36750720640382845 [[0.50243583]] [[-0.35489398]]
0.36641405564542384 [[0.50115982]] [[-0.35489398]]
0.36542808578669356 [[0.50000728]] [[-0.35489398]]
0.36453362012733137 [[0.49896036]] [[-0.35489398]]
0.3637180436570346 [[0.49800466]] [[-0.35489398]]
0.36297106523404715 [[0.49712839]] [[-0.35489398]]
0.3622841854376972 [[0.49632184]] [[-0.35489398]]
0.36165030544814103 [[0.49557683]] [[-0.35489398]]
0.36106343472639635 [[0.4948865]] [[-0.3